In [3]:
# Note to self add more comments and descriptions

class variable:
    'A class for handling all variables'
    
    # converting variable objects with all exponents as zero to floats
    def __new__(cls, co, names, exps):
        
        if [number for number in exps if number] and co:
            return super(variable, cls).__new__(cls)
        else:
            return float(co)
    
    def __init__(self, co, names, exps):
        self.co = co
        self.var = {}
        for i in range(len(names)):
            if exps[i] != 0:
                self.var[str(names[i])] = exps[i]  
        
    # Operator overloading
    
    # Mathematical Operations
    
    def __add__(self, other):
        if self.var == other.var:
            return variable(self.co + other.co, list(self.var.keys()), list(self.var.values()))
        else:
            return polynomial(self, other)
    
    def __radd__(self, other):
        return self + other
        
    def __sub__(self, other):
        return self + (-other)
    
    def __rsub__(self, other):
        return -self + other

    def __mul__(self, other):
        if type(other).__name__ == "variable":
            names = []
            exps = []
            
            for i in self.var:
                names += [i]
                exps += [self.var[i]]
                if i in other.var:
                    exps[-1] += other.var[i]
                    
            for i in other.var:
                if i not in names:
                    names += [i]
                    exps += [other.var[i]]
                    
            return variable(self.co * other.co, names, exps)
        
        else:
            return variable(self.co * other, list(self.var.keys()), list(self.var.values()))
        
    def __rmul__(self, other):
        return self*other

    
    def __truediv__(self, other):
        if type(other).__name__ == "variable":
            return self*variable(1/other.co, list(other.var.keys()), [-x for x in list(other.var.values())])
            
        else:
            return variable(self.co / other, list(self.var.keys()), list(self.var.values()))
        
    def __rtruediv__(self, other):
        return variable.__truediv__(other, self)
    
    def __pow__(self, other):
        if type(other).__name__ in ['int', 'float']:
            co = self.co**other
            names = list(self.var.keys())
            exps = [x**other for x in list(self.var.values())]
            return variable(co, names, exps)
        else:
            if self.co == 1:
                return variable(1, list(self.var.keys()), [exp*other for exp in self.var.values()])
            else:
                return variable(1, list(self.var.keys()) + [self.co], [exp*other for exp in self.var.values()] + [other])
    
    def __rpow__(self, other):
        return variable(1, [str(other)], [self])
    
    
    # Comparison Operations
    
    def __eq__(self, other):
        if type(other).__name__ == "variable":
            if self.co == other.co and self.var == other.var:
                return True
            else:
                return False
        
        else:
            return False

        
    def __ne__(self, other):
        if self == other:
            return False
        else:
            return True
        
    # The following operators are merely for sorting purposes 
            
    def __lt__(self, other):
        if type(other).__name__ in ['int', 'float']:
            return True
        else:
            selfnames = sorted(self.var.keys())
            othernames = sorted(other.var.keys())
            allnames = sorted(selfnames + othernames)
            if self == other:
                return False
            elif selfnames == othernames:
                for i in allnames:
                    if self.var[i] > other.var[i]:
                        return True
                    elif self.var[i] < other.var[i]:
                        return False

            else:
                for i in allnames:
                    if i in selfnames and i not in othernames:
                        return True
                    elif i not in selfnames and i in othernames:
                        return False
                
    def __le__(self, other):
        if self == other or self < other:
            return True
        else:
            return False

    def __gt__(self, other):
        if self == other or self < other:
            return False
        else:
            return True
                
    def __ge__(self, other):
        if self == other or self > other:
            return True
        else:
            return False
        
    # Evaluations
        
    def __repr__(self):
        if self.co != 1:
            printout = str(self.co) + '*'
        else:
            printout = ''
        keys = sorted(self.var.keys())
        for i in keys:
            printout += '(' + i + '**' + str(self.var[i]) + ')*'
        return printout[0:-1]

    
    def __str__(self):
        if self.co != 1:
            printout = str(self.co) + '*'
        else:
            printout = ''
        keys = sorted(self.var.keys())
        for i in keys:
            printout += '(' + i + '^' + '(' + str(self.var[i]) + ')' + ')*'
        return printout[0:-1]
    
    def __bool__(self):
        if self.co and self.var:
            return True
        else:
            return False
        
    def __neg__(self):
        return (-1)*self
    
    def __call__(self, data):
        "data is a dictionary of variables and their values"
        output = 1
        output *= self.co
        for key in self.var:
            if type(self.var[key]).__name__ in ['variable', 'polynomial']:
                try:
                    output *= data[key]**self.var[key](data)
                except KeyError:
                    output *= float(key)**self.var[key](data)
            else:
                try:
                    output *= data[key]**self.var[key]
                except KeyError:
                    output *= float(key)**self.var[key]
        return output
            
        

        
class polynomial:
    "Stores and handles multiple variables"
    
    def __new__(cls, *variables):     
        if len(variables) == 1:
            return variables[0]       
        else:
            return super(polynomial, cls).__new__(cls)
    
    def __init__(self, *variables):
        
        function = []
        for variable in variables:
            if variable not in function:
                function += [variable]
            else:
                function[function.index(variable)] += variable
        function = sorted(function)
        self.function = function             
        
    def __add__(self, other):
        if type(other).__name__ == "polynomial":
            return polynomial(*self.function, *other.function)
        else:
            return polynomial(*self.function, other)
        
    def __radd__(self, other):
        return self + other 
        
    def __sub__(self, other):
        if type(other).__name__ == "polynomial":
            return polynomial(*self.function, *[-x for x in other.function])
        else:
            return polynomial(*self.function, other*-1)

    def __rsub__(self, other):
        return (-self) + other

    def __mul__(self, other):
        if type(other).__name__ != "polynomial":
            return polynomial(*[x*other for x in self.function])
        else:
            functions = [(self*i).function for i in other.function]
            return polynomial(*[x for sublist in functions for x in sublist])
        
    def __rmul__(self, other):
        return self*other
    
    def __truediv__(self, other):
        if type(other).__name__ == "polynomial":
            return fraction(self, other)
        else:
            return polynomial(*[x/other for x in self.function])
        
    def __rtruediv__(self, other):
        return fraction(other, self)
        
    def __pow__(self, other):
        if type(other).__name__ in ['int']:
            output = 1
            output *= self
            for i in range(1, other):
                output *= self
            return output
        else:
            return None

    def __rpow__(self, other):
        return variable(1, [other], [self])
        
    
    def __bool__(self):
        for element in self.function:
            if element:
                return True
        return False
        
    
    def __eq__(self, other):
        if type(other).__name__ == "polynomial":
            if self.function == other.function:
                return True
            else:
                return False
        
        else:
            return False
        

        
    def __ne__(self, other):
        if self == other:
            return False
        else:
            return True
            
    
    def __str__(self):
        string = str(self.function[0])
        for i in range(1, len(self.function)):
            string += " + " + str(self.function[i])
        return string
    
    def __len__(self):
        return len(self.function)
    
    def __neg__(self):
        return (-1)*self
    
    def __call__(self, data):
        "data is a dictionary of variables and their values"
        output = 0
        for variable in self.function:
            output += variable(data)
        return output
    
class fraction:
    "Handles Polynomial fraction"
    
    def __new__(cls, numerator, denominator):     
        if numerator == denominator:
            return int(1)       
        elif type(denominator).__name__ in ['int', 'float', 'variable', 'fraction'] \
        and (type(numerator).__name__ != 'int' or type(denominator).__name__ != 'int'):
            return numerator/denominator
        else:
            return super(fraction, cls).__new__(cls)
    
    def __init__(self, numerator, denominator):
        self.num = numerator
        self.den = denominator
    
    def __add__(self, other):
        if self.den == other.den:
            return fraction(self.num + other.num, self.den)
        else:
            return Polynomial(self, other)
        
    def __radd__(self, other):
        return self + other
        
    def __sub__(self, other):
        return self + (-other)
    
    def __rsub__(self, other):
        return (-self) + other
    
    def __mul__(self, other):
        if type(other).__name__ == "fraction":
            return fraction(self.num*other.num, self.den*other.den)
        elif type(other).__name__ in ['variable', 'int', 'float']:
            return fraction(self.num*other, self.den)
        else:
            return None
        
    def __rmul__(self, other):
        return self*other
        
    def __truediv__(self, other):
        if type(other).__name__ == "fraction":
            return fraction(self.num*other.den, self.den*other.num)
        else:
            return fraction(self.num, self.den*other)
        
    def __rtruediv__(self, other):
        return fraction(self.den, self.num)*other
        
    def __pow__(self, other):
        return fraction(self.num**other, self.den**other)
    
    def __rpow__(self, other):
        return variable(1, [other], [self])
    
    def __eq__(self, other):
        if type(other).__name__ == "fraction":
            if self.num == other.num and self.den == other.den:
                return True
            else:
                return False
        
        elif [type(self.num).__name__, type(self.den).__name__] \
        == ['int', 'int'] and type(other).__name__ in ['int', 'float']:
            
            if self.num/self.den == other:
                return True
            else:
                return False
            
        else:
            return False
        
  
    def __ne__(self, other):
        if self == other:
            return False
        else:
            return True
        
    def __str__(self):
        return '(' + str(self.num) + ')' + '/' + '(' + str(self.den) + ')'
    
    def __neg__(self):
        return self*(-1)
    
    def __call__(self, data=None):
        "data is a dictionary of variables and their values"
        if [type(self.num).__name__, type(self.den).__name__] == ['int', 'int']:
            return self.num/self.den
        else:
            return self.num(data)/self.den(data)

        
        
# Add more methods to fractions and polynimial classes eg reflected operations and exponentiation
# variable declaration is as follows: variable(constant, variables, exponenets)
# result is constant*(variable1**exponent1)*(variable2**exponenet2)*...

x3y3 = variable(1, ['x', 'y'], [2, 2])
y2z4 = variable(1, ['y', 'z'], [2, 1])


x = variable(1, ['x'], [2])
y = variable(1, ['y'], [3])
data = {'x':1, 'y':2, 'z':1}
print((2**(x3y3**y2z4))(data))
print((x+y)(data))

1.157920892373162e+77
4
4
9


In [35]:
x = variable(1, ['x', 'y'], [1])
y = variable(2, ['y'], [2])
data = {'x':2, 'y':3}
print(1/(x+y))

print(fraction(x, 3))


#else:
#    raise TypeError("Equality between object of type "
#                    + "'" + type(self).__name__ + "'" + " and " 
#                    + "'" + type(other).__name__ + "'" + " is unclear")

(1)/((x^(1)) + 2*(y^(2)))
0.3333333333333333*(x^(1))
